In [1]:
from src.chat import Chat
from app.models.UserInput import UserInput

In [2]:
chat = Chat()
chat.init_app()

/Users/gcpuppy/Documents/WorkSpace/DataSci/dsde-project/src/database/qdrant.py:37: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  self.df = pd.read_csv("combined_data.csv")


## Clear Chat

In [9]:
chat_id = "abc"
chat.clear_chat(chat_id=chat_id)

## Set System Prompt

In [ ]:
user_message = "Give me the papers that related to field of aerodynamic especially the military purposes"

user_input = UserInput(
    chat_id=chat_id,
    message=user_message,
    currentGraph=None
)

# Goal
1. Give "Keyword node"
2. Give "GrapLink from GraphData"

In [4]:
### FIX HERE
system_template_prompt = """You are an AI assistance for encouraging the system to initialize new "Node" from below data that was fetched from QdrantDB. Node have two types which are "Paper" and "Keyword".
Keyword is a short word that related to topic, title or field of the input messages from user while "Paper" is the main content of articles in the database. Your tasks are generating "Keyword" node based on summarizing the content of user messages and "Paper" node, a node that contains more . Both of them follow this structure of node object: 
export interface Node {
title: string
type: 'paper' | 'keyword'
color?: string
citations?: number
year?: number
abstract?: string
authors?: string[]
relevance?: number
}
export interface GraphLink {
source: string
target: string
index?: number
}
Don't forget to ensure that the values that will be assigned are related to data in the databas
This is the guideines for you:
1. Analyze the given user input "Keyword".
2. Create nodes based on related keywords.
3. Give me the response in format of Json of GraphLink
source is the ID of starting node,
target is ID of destination node,
"""
### FIX HERE

chat.set_system_template_prompt(system_template_prompt)

# Set Chat Template Prompt

In [15]:
### FIX HERE
chat_template_prompt = """
Below is the GraphData that was fetched from QdrantDB. You can use this data to generate new "Node" based on the user message.
{paper_data}
"""
### FIX HERE

chat.set_chat_template_prompt(chat_template_prompt)

## Example of {paper_data}
====================================================================================================================================

```json
{
    "id": "2410.18541_arxiv",
    "title": "On Explaining with Attention Matrices",
    "type": "paper",
    "year": 2024,
    "abstract": "  This paper explores the much discussed, possible explanatory link between\nattention weights (AW) in transformer models and predicted output. Contrary to\nintuition and early research on attention, more recent prior research has\nprovided formal arguments and empirical evidence that AW are not explanatorily\nrelevant. We show that the formal arguments are incorrect. We introduce and\neffectively compute efficient attention, which isolates the effective\ncomponents of attention matrices in tasks and models in which AW play an\nexplanatory role. We show that efficient attention has a causal role (provides\nminimally necessary and sufficient conditions) for predicting model output in\nNLP tasks requiring contextual information, and we show, contrary to [7], that\nefficient attention matrices are probability distributions and are effectively\ncalculable. Thus, they should play an important part in the explanation of\nattention based model behavior. We offer empirical experiments in support of\nour method illustrating various properties of efficient attention with various\nmetrics on four datasets.\n",
    "authors": [
        "Omar Naim and Nicholas Asher"
    ],
    "source": "arxiv"
}
```

==========================================================================================================
```json
{
    "id": "2204.13154_arxiv",
    "title": "Attention Mechanism in Neural Networks: Where it Comes and Where it Goes",
    "type": "paper",
    "year": 2022,
    "abstract": "  A long time ago in the machine learning literature, the idea of incorporating\na mechanism inspired by the human visual system into neural networks was\nintroduced. This idea is named the attention mechanism, and it has gone through\na long development period. Today, many works have been devoted to this idea in\na variety of tasks. Remarkable performance has recently been demonstrated. The\ngoal of this paper is to provide an overview from the early work on searching\nfor ways to implement attention idea with neural networks until the recent\ntrends. This review emphasizes the important milestones during this progress\nregarding different tasks. By this way, this study aims to provide a road map\nfor researchers to explore the current development and get inspired for novel\napproaches beyond the attention.\n",
    "authors": [
        "Derya Soydaner"
    ],
    "source": "arxiv"
}
```

## Example output

```json
{
    "GraphNodes" : [
        {
            data : {
                "id": "2410.18541_arxiv",
                "title": "On Explaining with Attention Matrices",
                "type": "paper",
                "year": 2024,
                "abstract": "  This paper explores the much discussed, possible explanatory link between\nattention weights (AW) in transformer models and predicted output. Contrary to\nintuition and early research on attention, more recent prior research has\nprovided formal arguments and empirical evidence that AW are not explanatorily\nrelevant. We show that the formal arguments are incorrect. We introduce and\neffectively compute efficient attention, which isolates the effective\ncomponents of attention matrices in tasks and models in which AW play an\nexplanatory role. We show that efficient attention has a causal role (provides\nminimally necessary and sufficient conditions) for predicting model output in\nNLP tasks requiring contextual information, and we show, contrary to [7], that\nefficient attention matrices are probability distributions and are effectively\ncalculable. Thus, they should play an important part in the explanation of\nattention based model behavior. We offer empirical experiments in support of\nour method illustrating various properties of efficient attention with various\nmetrics on four datasets.\n",
                "authors": [
                    "Omar Naim and Nicholas Asher"
                ],
                "source": "arxiv"
            }
        },
        {
            data : {
                "id": "2204.13154_arxiv",
                "title": "Attention Mechanism in Neural Networks: Where it Comes and Where it Goes",
                "type": "paper",
                "year": 2022,
                "abstract": "  A long time ago in the machine learning literature, the idea of incorporating\na mechanism inspired by the human visual system into neural networks was\nintroduced. This idea is named the attention mechanism, and it has gone through\na long development period. Today, many works have been devoted to this idea in\na variety of tasks. Remarkable performance has recently been demonstrated. The\ngoal of this paper is to provide an overview from the early work on searching\nfor ways to implement attention idea with neural networks until the recent\ntrends. This review emphasizes the important milestones during this progress\nregarding different tasks. By this way, this study aims to provide a road map\nfor researchers to explore the current development and get inspired for novel\napproaches beyond the attention.\n",
                "authors": [
                    "Derya Soydaner"
                ],
                "source": "arxiv"
            }
        },
        {
            // Keyword Node added
            data: {
                "id" : "transformer-and-attention-is-all-you-need",
                "title" : "Transformer and Attention is all you need",
                "type" : "keyword",
                "abstract" : "",
                "authors" : "",
                "source" : ""
            }
        }
    ],
    "GraphLinks" : [
        {
            "source" : "2410.18541_arxiv",
            "target" : "2204.13154_arxiv"
        },
        {
            "source" : "transformer-and-attention-is-all-you-need",
            "target" : "2204.13154_arxiv"
        },
        {
            "source" : "transformer-and-attention-is-all-you-need",
            "target" : "2410.18541_arxiv"
        },
    ]
}
```

In [6]:
### See the result of the chat

chat.test_initial_chat(user_input=user_input)

================================ Human Message =================================


{
    "id": "2410.18541_arxiv",
    "title": "On Explaining with Attention Matrices",
    "type": "paper",
    "year": 2024,
    "abstract": "  This paper explores the much discussed, possible explanatory link between\nattention weights (AW) in transformer models and predicted output. Contrary to\nintuition and early research on attention, more recent prior research has\nprovided formal arguments and empirical evidence that AW are not explanatorily\nrelevant. We show that the formal arguments are incorrect. We introduce and\neffectively compute efficient attention, which isolates the effective\ncomponents of attention matrices in tasks and models in which AW play an\nexplanatory role. We show that efficient attention has a causal role (provides\nminimally necessary and sufficient conditions) for predicting model output in\nNLP tasks requiring contextual information, and we show, contrary to [7], that\nef

# Set Detect Additional Data

In [7]:
detect_additional_data_template = """
{user_message}

{current_graph}
"""
chat.set_detect_additional_data_template(detect_additional_data_template)